In [545]:
import torch
import data_loader
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [546]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [547]:
# helper.py

# Count the number of parameters
def count_param(model):
    return sum([torch.numel(param) for param in model.parameters()])

In [548]:
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=42)

In [549]:
def compute_nb_errors(model, data_loader):

    nb_data_errors = 0

    for data_input, data_target, data_classes in data_loader:
        data_target = torch.nn.functional.one_hot(data_target)
        output = model(data_input)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [550]:
def compute_nb_errors_siamese(model, data_loader):

    nb_data_errors = 0
    for data_input, data_target, data_classes in data_loader:
        data_1, data_2 = data_input.unbind(1)               
        output = model(data_1.unsqueeze(1), data_2.unsqueeze(1))
        data_target = torch.nn.functional.one_hot(data_target)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [676]:
def compute_nb_errors_auxsiamese(model, data_loader):

    nb_data_errors = 0
    for data_input, data_target, data_classes in data_loader:
        data_1, data_2 = data_input.unbind(1)               
        output, aux1, aux2 = model(data_1.unsqueeze(1), data_2.unsqueeze(1))
        data_target = torch.nn.functional.one_hot(data_target)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [637]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(10, 2)
        
    def forward(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        x = x.view(-1, 256) # size [nb, 256]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [639]:
class SiameseBaseNet(nn.Module):
    def __init__(self):
        super(SiameseBaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc1(x1)))
        x1 = F.relu(self.fc2(x1))
        
        x2 = self.convs(x2)
        x2 = x2.view(-1, 256)
        x2 = F.relu(self.fc1(x2))
        x2 = F.relu(self.fc2(x2))
        
        x = torch.cat([x1, x2], dim=1)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x

In [673]:
class AuxsiameseBaseNet(nn.Module):
    def __init__(self):
        super(AuxsiameseBaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc1(x1)))
        x1 = F.relu(self.fc2(x1))
        
        x2 = self.convs(x2)
        x2 = x2.view(-1, 256)
        x2 = F.relu(self.fc1(x2))
        x2 = F.relu(self.fc2(x2))
        
        x = torch.cat([x1, x2], dim=1)
        aux1 = F.softmax(x1)
        aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x, aux1, aux2

In [699]:
class AuxBaseNet(nn.Module):
    def __init__(self):
        super(AuxBaseNet, self).__init__()
        self.conv11 = nn.Conv2d(1, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv21 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc11 = nn.Linear(256, 200)
        self.fc21 = nn.Linear(200, 10)
        self.conv12 = nn.Conv2d(1, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv22 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc12 = nn.Linear(256, 200)
        self.fc22 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = F.relu(F.max_pool2d(self.conv11(x1), kernel_size=2)) # size [nb, 32, 5, 5]  
        x1 = F.relu(F.max_pool2d(self.conv21(x1), kernel_size=2)) # size [nb, 64, 2, 2]
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc11(x1)))
        x1 = F.relu(self.fc21(x1))
        
        x2 = F.relu(F.max_pool2d(self.conv12(x2), kernel_size=2)) # size [nb, 32, 5, 5]  
        x2 = F.relu(F.max_pool2d(self.conv22(x2), kernel_size=2)) # size [nb, 64, 2, 2]
        x2 = x2.view(-1, 256)
        x2 = F.relu((self.fc12(x2)))
        x2 = F.relu(self.fc22(x2))
        
        x = torch.cat([x1, x2], dim=1)
        aux1 = F.softmax(x1)
        aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x, aux1, aux2

In [733]:
model = BaseNet()

# Calculate the number of parameters in the model
count_param(model)

63320

In [734]:
model = SiameseBaseNet()

# Calculate the number of parameters in the model
count_param(model)

62540

In [735]:
model = AuxsiameseBaseNet()

# Calculate the number of parameters in the model
count_param(model)

62540

In [736]:
model = AuxBaseNet()

# Calculate the number of parameters in the model
count_param(model)

125038

In [702]:
def train(model, train_loader, eta, decay, n_epochs=25, verbose=False, siamese=False, aux=False, alpha = 0):

    #binary_crit = nn.CrossEntropyLoss()
    binary_crit = torch.nn.BCELoss()
    aux_crit = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=eta, weight_decay=decay)
    
    tr_losses = []
    tr_accuracies = []

    for e in range(n_epochs):
        # Reset training/validation loss
        tr_loss = 0

        # Training model
        model.train()

        for train_input, train_target, train_classes in iter(train_loader):
            train_target = torch.nn.functional.one_hot(train_target)
            # Forward pass
            
            if siamese == True:
                train_1, train_2 = train_input.unbind(1)
                if aux == True:
                    output, aux1, aux2 = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
                else:
                    output = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
            elif aux == True:
                train_1, train_2 = train_input.unbind(1)
                output, aux1, aux2 = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
            else:
                output = model(train_input)
                
            # Binary classification loss
            binary_loss = binary_crit(output, train_target.float())
            total_loss = binary_loss
            
            # Auxiliary loss
            if aux == True:

                aux_loss1 = aux_crit(aux1, train_classes[:,0])
                aux_loss2 = aux_crit(aux2, train_classes[:,1])
                aux_loss = aux_loss1 + aux_loss2
                total_loss = binary_loss + aux_loss * alpha
        
            # Total loss = Binary loss + aux loss * alpha
            
            tr_loss += total_loss

            # Backward pass
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

        # Collect accuracy data
        # tr_accuracies.append(compute_nb_errors_siamese(model, train_loader)/1000)

        # Collect loss data
        tr_losses.append(tr_loss)

        if verbose:
            print('Epoch %d/%d, Binary loss: %.3f' %
                  (e+1, n_epochs, tr_loss))

In [642]:
import time

time1 = time.perf_counter()
model = SiameseBaseNet()
#model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 13.576
Epoch 2/25, Binary loss: 11.614
Epoch 3/25, Binary loss: 9.312
Epoch 4/25, Binary loss: 7.733
Epoch 5/25, Binary loss: 6.548
Epoch 6/25, Binary loss: 5.634
Epoch 7/25, Binary loss: 4.718
Epoch 8/25, Binary loss: 3.844
Epoch 9/25, Binary loss: 2.893
Epoch 10/25, Binary loss: 2.243
Epoch 11/25, Binary loss: 1.638
Epoch 12/25, Binary loss: 1.064
Epoch 13/25, Binary loss: 0.753
Epoch 14/25, Binary loss: 0.469
Epoch 15/25, Binary loss: 0.279
Epoch 16/25, Binary loss: 0.192
Epoch 17/25, Binary loss: 0.136
Epoch 18/25, Binary loss: 0.103
Epoch 19/25, Binary loss: 0.081
Epoch 20/25, Binary loss: 0.065
Epoch 21/25, Binary loss: 0.054
Epoch 22/25, Binary loss: 0.046
Epoch 23/25, Binary loss: 0.040
Epoch 24/25, Binary loss: 0.035
Epoch 25/25, Binary loss: 0.031
Spend 1.018782e+01 s
tensor(1.) tensor(0.8500)


In [651]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 13.564
Epoch 2/25, Binary loss: 12.049
Epoch 3/25, Binary loss: 10.678
Epoch 4/25, Binary loss: 9.086
Epoch 5/25, Binary loss: 7.613
Epoch 6/25, Binary loss: 6.530
Epoch 7/25, Binary loss: 5.443
Epoch 8/25, Binary loss: 4.033
Epoch 9/25, Binary loss: 2.860
Epoch 10/25, Binary loss: 1.917
Epoch 11/25, Binary loss: 1.299
Epoch 12/25, Binary loss: 0.851
Epoch 13/25, Binary loss: 0.497
Epoch 14/25, Binary loss: 0.293
Epoch 15/25, Binary loss: 0.174
Epoch 16/25, Binary loss: 0.119
Epoch 17/25, Binary loss: 0.092
Epoch 18/25, Binary loss: 0.074
Epoch 19/25, Binary loss: 0.060
Epoch 20/25, Binary loss: 0.051
Epoch 21/25, Binary loss: 0.043
Epoch 22/25, Binary loss: 0.039
Epoch 23/25, Binary loss: 0.032
Epoch 24/25, Binary loss: 0.028
Epoch 25/25, Binary loss: 0.026
Spend 5.444853e+00 s
tensor(1.) tensor(0.8280)


In [687]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxsiameseBaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

<ipython-input-673-21efc383ef87>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-673-21efc383ef87>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Epoch 1/25, Binary loss: 41.301
Epoch 2/25, Binary loss: 40.527
Epoch 3/25, Binary loss: 39.194
Epoch 4/25, Binary loss: 37.589
Epoch 5/25, Binary loss: 35.413
Epoch 6/25, Binary loss: 33.461
Epoch 7/25, Binary loss: 32.224
Epoch 8/25, Binary loss: 31.236
Epoch 9/25, Binary loss: 30.435
Epoch 10/25, Binary loss: 29.611
Epoch 11/25, Binary loss: 29.025
Epoch 12/25, Binary loss: 28.527
Epoch 13/25, Binary loss: 28.140
Epoch 14/25, Binary loss: 27.689
Epoch 15/25, Binary loss: 27.367
Epoch 16/25, Binary loss: 27.219
Epoch 17/25, Binary loss: 27.102
Epoch 18/25, Binary loss: 26.959
Epoch 19/25, Binary loss: 26.890
Epoch 20/25, Binary loss: 26.829
Epoch 21/25, Binary loss: 26.785
Epoch 22/25, Binary loss: 26.759
Epoch 23/25, Binary loss: 26.788
Epoch 24/25, Binary loss: 26.739
Epoch 25/25, Binary loss: 26.726
Spend 1.036312e+01 s
tensor(1.) tensor(0.8700)


In [712]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxBaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

<ipython-input-699-f890ddbc2f26>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-699-f890ddbc2f26>:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Epoch 1/25, Binary loss: 41.354
Epoch 2/25, Binary loss: 40.233
Epoch 3/25, Binary loss: 38.545
Epoch 4/25, Binary loss: 36.119
Epoch 5/25, Binary loss: 34.638
Epoch 6/25, Binary loss: 32.981
Epoch 7/25, Binary loss: 32.049
Epoch 8/25, Binary loss: 31.040
Epoch 9/25, Binary loss: 30.221
Epoch 10/25, Binary loss: 29.890
Epoch 11/25, Binary loss: 29.816
Epoch 12/25, Binary loss: 28.891
Epoch 13/25, Binary loss: 28.312
Epoch 14/25, Binary loss: 27.903
Epoch 15/25, Binary loss: 27.736
Epoch 16/25, Binary loss: 27.181
Epoch 17/25, Binary loss: 26.733
Epoch 18/25, Binary loss: 26.162
Epoch 19/25, Binary loss: 26.032
Epoch 20/25, Binary loss: 25.755
Epoch 21/25, Binary loss: 25.475
Epoch 22/25, Binary loss: 25.369
Epoch 23/25, Binary loss: 25.333
Epoch 24/25, Binary loss: 25.143
Epoch 25/25, Binary loss: 25.044
Spend 9.934680e+00 s
tensor(1.) tensor(0.8390)


In [727]:
accuracies = []
times = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    model = BaseNet()
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False)
    time2 = time.perf_counter()
    times.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies.append(te_accuracy)

print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies).mean(), torch.tensor(accuracies).std()))

tensor(1.) tensor(0.8210)
tensor(1.) tensor(0.8240)
tensor(1.) tensor(0.8120)
tensor(1.) tensor(0.8180)
tensor(0.9990) tensor(0.8260)
tensor(1.) tensor(0.8080)
tensor(1.) tensor(0.8110)
tensor(0.9990) tensor(0.8330)
tensor(1.) tensor(0.8210)
tensor(1.) tensor(0.8020)
tensor(1.) tensor(0.8190)
tensor(1.) tensor(0.8220)
tensor(1.) tensor(0.8370)
tensor(1.) tensor(0.8180)
tensor(1.) tensor(0.8280)
tensor(1.) tensor(0.8410)
tensor(1.) tensor(0.8280)
tensor(1.) tensor(0.8300)
tensor(0.9990) tensor(0.8240)
tensor(1.) tensor(0.8010)
Mean: 0.821, Std: 0.011


In [728]:
accuracies1 = []
times1 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    model = SiameseBaseNet()
    #model = BaseNet()
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True)
    time2 = time.perf_counter()
    times1.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies1.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies1).mean(), torch.tensor(accuracies1).std()))

tensor(1.) tensor(0.8410)
tensor(1.) tensor(0.8490)
tensor(1.) tensor(0.8380)
tensor(1.) tensor(0.8480)
tensor(1.) tensor(0.8600)
tensor(1.) tensor(0.8450)
tensor(1.) tensor(0.8590)
tensor(1.) tensor(0.8410)
tensor(1.) tensor(0.8490)
tensor(1.) tensor(0.8570)
tensor(1.) tensor(0.8600)
tensor(1.) tensor(0.8430)
tensor(1.) tensor(0.8550)
tensor(1.) tensor(0.8360)
tensor(1.) tensor(0.8530)
tensor(1.) tensor(0.8470)
tensor(1.) tensor(0.8630)
tensor(1.) tensor(0.8550)
tensor(1.) tensor(0.8600)
tensor(1.) tensor(0.8500)
Mean: 0.850, Std: 0.008


In [731]:
accuracies2 = []
times2 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxBaseNet()
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.8)
    time2 = time.perf_counter()
    times2.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies2.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies2).mean(), torch.tensor(accuracies2).std()))

<ipython-input-699-f890ddbc2f26>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-699-f890ddbc2f26>:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


tensor(0.9980) tensor(0.8340)
tensor(0.9870) tensor(0.8420)
tensor(0.9980) tensor(0.8400)
tensor(0.9950) tensor(0.8120)
tensor(1.) tensor(0.8480)
tensor(0.9970) tensor(0.8580)
tensor(0.9800) tensor(0.8440)
tensor(1.) tensor(0.8550)
tensor(0.9980) tensor(0.8630)
tensor(0.9990) tensor(0.8500)
tensor(0.9840) tensor(0.8540)
tensor(0.9970) tensor(0.8350)
tensor(1.) tensor(0.8450)
tensor(0.9980) tensor(0.8360)
tensor(0.9790) tensor(0.8440)
tensor(1.) tensor(0.8730)
tensor(0.9960) tensor(0.8250)
tensor(1.) tensor(0.8390)
tensor(0.9980) tensor(0.8430)
tensor(0.9940) tensor(0.8150)
Mean: 0.843, Std: 0.015


In [732]:
accuracies3 = []
times3 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxsiameseBaseNet()
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = 0.8)
    time2 = time.perf_counter()
    times3.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies3.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies3).mean(), torch.tensor(accuracies3).std()))

<ipython-input-673-21efc383ef87>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-673-21efc383ef87>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


tensor(1.) tensor(0.8550)
tensor(0.9990) tensor(0.8650)
tensor(0.9970) tensor(0.8350)
tensor(0.9980) tensor(0.8560)
tensor(0.9990) tensor(0.8670)
tensor(0.9990) tensor(0.8540)
tensor(0.9990) tensor(0.8580)
tensor(1.) tensor(0.8560)
tensor(1.) tensor(0.8590)
tensor(1.) tensor(0.8530)
tensor(0.9910) tensor(0.8660)
tensor(1.) tensor(0.8630)
tensor(0.9920) tensor(0.8740)
tensor(1.) tensor(0.8440)
tensor(1.) tensor(0.8570)
tensor(0.9870) tensor(0.8540)
tensor(0.9980) tensor(0.8770)
tensor(0.9890) tensor(0.8290)
tensor(1.) tensor(0.8590)
tensor(0.9840) tensor(0.8520)
Mean: 0.857, Std: 0.011


In [419]:
gammas = torch.logspace(start=-4, end=-2, steps=5)
decays = torch.logspace(start=-13, end=-8, steps=6)
accuracies = torch.empty((len(gammas), len(decays)))
model = BaseNet()
for j in range(len(gammas)):
    for k in range(len(decays)):
        accurate = []
        for i in range(10):
            train_loader, test_loader = load_data(N=1000, batch_size=50, seed=42)
            train(model, train_loader, gammas[j], decays[k], 25, verbose=False)
            te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
            accurate.append(te_accuracy)
        accuracies[j,k] = torch.Tensor(accurate).mean()

KeyboardInterrupt: 

In [713]:
for j in range(10):
    accuracies3 = []
    times3 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
        time1 = time.perf_counter()
        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxBaseNet()
        train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)
        time2 = time.perf_counter()
        times3.append(time2 - time1)

        tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies3.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies3).mean(), torch.tensor(accuracies3).std()))

<ipython-input-699-f890ddbc2f26>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-699-f890ddbc2f26>:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Mean: 0.838, Std: 0.009
Mean: 0.839, Std: 0.011
Mean: 0.837, Std: 0.012
Mean: 0.844, Std: 0.012
Mean: 0.840, Std: 0.011
Mean: 0.841, Std: 0.012
Mean: 0.838, Std: 0.014
Mean: 0.843, Std: 0.016
Mean: 0.846, Std: 0.014
Mean: 0.846, Std: 0.020


In [714]:
for j in range(10):
    accuracies2 = []
    times2 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
        time1 = time.perf_counter()
        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxsiameseBaseNet()
        train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = j/10)
        time2 = time.perf_counter()
        times2.append(time2 - time1)

        tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies2.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies2).mean(), torch.tensor(accuracies2).std())) 

<ipython-input-673-21efc383ef87>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-673-21efc383ef87>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Mean: 0.849, Std: 0.007
Mean: 0.851, Std: 0.008
Mean: 0.852, Std: 0.011
Mean: 0.852, Std: 0.012
Mean: 0.852, Std: 0.009
Mean: 0.850, Std: 0.013
Mean: 0.854, Std: 0.007
Mean: 0.842, Std: 0.031
Mean: 0.857, Std: 0.009
Mean: 0.857, Std: 0.007


In [715]:
for j in range(10,20):
    accuracies2 = []
    times2 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
        time1 = time.perf_counter()
        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxsiameseBaseNet()
        train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = j/10)
        time2 = time.perf_counter()
        times2.append(time2 - time1)

        tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies2.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies2).mean(), torch.tensor(accuracies2).std())) 

<ipython-input-673-21efc383ef87>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux1 = F.softmax(x1)
<ipython-input-673-21efc383ef87>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  aux2 = F.softmax(x2)


Mean: 0.851, Std: 0.013
Mean: 0.852, Std: 0.023
Mean: 0.851, Std: 0.021
Mean: 0.852, Std: 0.019
Mean: 0.855, Std: 0.021
Mean: 0.849, Std: 0.027
Mean: 0.843, Std: 0.038
Mean: 0.835, Std: 0.054
Mean: 0.852, Std: 0.036
Mean: 0.830, Std: 0.054
